[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-fast-colab/blob/main/stable_fast_colab.ipynb)

In [ ]:
# !pip install -q https://download.pytorch.org/whl/cu118/torchaudio-2.1.0%2Bcu118-cp310-cp310-linux_x86_64.whl
# !pip install -q https://download.pytorch.org/whl/torchdata-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q https://download.pytorch.org/whl/torchtext-0.16.0%2Bcpu-cp310-cp310-linux_x86_64.whl
# !pip install -q https://download.pytorch.org/whl/cu118/torchvision-0.16.0%2Bcu118-cp310-cp310-linux_x86_64.whl
# !pip install -q fastai==2.7.13
!pip install -q diffusers transformers accelerate
!pip install -q https://download.pytorch.org/whl/cu118/torch-2.1.0%2Bcu118-cp310-cp310-linux_x86_64.whl
!pip install -q https://download.pytorch.org/whl/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q https://github.com/camenduru/stable-fast/releases/download/colab/stable_fast-0.0.2-cp310-cp310-linux_x86_64.whl

# !pip install -q ninja
# !pip install -q git+https://github.com/chengzeyi/stable-fast.git@main#egg=stable-fast

# !apt -y update -qq
# !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
# %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!ldconfig /usr/lib64-nvidia

In [ ]:
import torch, xformers, triton
from diffusers import StableDiffusionPipeline
from sfast.compilers.stable_diffusion_pipeline_compiler import (compile, CompilationConfig)
torch.backends.cuda.matmul.allow_tf32 = True
model = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', torch_dtype=torch.float16, safety_checker=None).to('cuda')
config = CompilationConfig.Default()
config.enable_xformers = True
config.enable_triton = True
config.enable_cuda_graph = True # After capturing, the model only accepts one fixed image size. If you want the model to be dynamic, don't enable it.
compiled_model = compile(model, config)

In [ ]:
kwarg_inputs = dict(
    prompt=
    '(masterpiece:1,2), best quality, masterpiece, best detail face, lineart, monochrome, a beautiful girl',
    height=512,
    width=512,
    num_inference_steps=50,
    num_images_per_prompt=1,
)
output_image = compiled_model(**kwarg_inputs).images[0] # The first call will trigger compilation and might be very slow. # After the first call, it should be very fast.
output_image

In [ ]:
output_image = compiled_model(**kwarg_inputs).images[0]
output_image